# **Data Mining And Wrangling **

### **Data source:** *https://www.imdb.com/chart/top/*
####  Used different methods to get the get the data but we concluded with the *Top 250 Movies* from IMDB using Scrapy and spider

##### Load the data

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import missingno as mns 
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import requests
import json
from datetime import date
from collections import Counter
from wordcloud import WordCloud


### Since we saved the dataset as a JSON file, We had to convert it to csv format removing the the [] and [[]] formats (lists and nested lists)

In [1]:

# Load the JSON file
json_file_path = "D:\Scraping_IMDB_250_movies_using_Scrapy\imdbscrapper\IMDB_250_movies.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Convert list values to plain values
for movie in data:
    for key, value in movie.items():
        if isinstance(value, list):
            if all(isinstance(i, list) for i in value):  # If nested lists exist, flatten them
                movie[key] = ", ".join([", ".join(map(str, sublist)) for sublist in value])
            else:
                movie[key] = ", ".join(map(str, value))

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as CSV
csv_file_path = "IMDB_250_movies.csv"
df.to_csv(csv_file_path, index=False, encoding="utf-8")

csv_file_path


<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\EMMANUEL NSUBUGA\AppData\Local\Temp\ipykernel_3292\3240547807.py:2: SyntaxWarning: invalid escape sequence '\S'
  json_file_path = "D:\Scraping_IMDB_250_movies_using_Scrapy\imdbscrapper\IMDB_250_movies.json"
C:\Users\EMMANUEL NSUBUGA\AppData\Local\Temp\ipykernel_3292\3240547807.py:2: SyntaxWarning: invalid escape sequence '\S'
  json_file_path = "D:\Scraping_IMDB_250_movies_using_Scrapy\imdbscrapper\IMDB_250_movies.json"


NameError: name 'json' is not defined

#### Reading the csv Dataset 

In [ ]:
## reading the csv dataset 
df = pd.read_csv("IMDB_250_movies.csv")
df.head(6)

## **Exploratory Data Analysis**

In [ ]:
## checking the shape of the dataset for information
df.info()

### Now we can use the OMDB API to get the other attributes of the dataset i.e. Main Actors, Budget, Box Office Gross, and Awards

In [ ]:
# OMDb API Key
OMDB_API_KEY = "60fded84"

# Function to fetch movie details from OMDb API
def fetch_movie_details(title):
    url = f"http://www.omdbapi.com/?t={title}&apikey={OMDB_API_KEY}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Check if the response is valid
        if data.get("Response") == "True":
            return {
                "title": data.get("Title", title),  # Ensure title matches
                "Cast (Main Actors)": data.get("Actors", "N/A"),
                "Budget": "N/A",  # OMDb does not provide budget info
                "Box Office Gross": data.get("BoxOffice", "N/A"),
                "Awards": data.get("Awards", "N/A")
            }
    
    # If no valid response, return empty data
    return {
        "title": title,
        "Cast (Main Actors)": "N/A",
        "Budget": "N/A",
        "Box Office Gross": "N/A",
        "Awards": "N/A"
    }

# Load your existing CSV file
existing_csv = "IMDB_250_movies.csv"
df = pd.read_csv(existing_csv)

# Fetch details for each movie
additional_data = [fetch_movie_details(title) for title in df["title"]]

# Convert the fetched data into a DataFrame
extra_df = pd.DataFrame(additional_data)

# Merge with the existing DataFrame
df = df.merge(extra_df, on="title", how="left")

# Save updated data
new_df_path = "Final_IMDB_250_movies_with_OMDB.csv"
df.to_csv(new_df_path, index=False, encoding="utf-8")

print(f"Updated CSV saved at: {new_df_path}")


## **Cleaning Data** 

In [ ]:
df1 = pd.read_csv("Final_IMDB_250_movies_with_OMDB.csv")

df1.columns


## **Visualizing the Missing Values**

In [ ]:
#### checking for missing values in the Box Office column
mns.bar(df1)

In [ ]:
#### checking for missing values in the Box Office column
missing_values = df1.isnull().sum()

missing_values

In [ ]:
### Dropping the Budget column
#The Budget column is not necessary for the analysis since it is only missing values, so we can drop it from the data frame.

df2 = df1.drop(columns=["Budget"])

df2.columns


In [ ]:
## checking the missing values in the dataset

df2.isnull().sum()

In [ ]:
### Dropping the missing values
#Since the missing values are few, we can drop them from the dataset.
df3 = df2.dropna()



In [ ]:
df3.head(6)

In [ ]:
# checking the missing values in the dataset again
df3.isna().sum()


## **Now we can save the data as a new dataset**

In [ ]:
# saving df3 to a new file
df3.to_csv("IMDB_250_movies_cleaned2.csv", index=False)


In [ ]:
### Loading the cleaned data
cleaned_data = pd.read_csv('IMDB_250_movies_cleaned2.csv')
cleaned_data.head(4)

### Converting the release_day, month, and year to strings

In [ ]:
cleaned_data[['release_day','release_month','release_year']] = cleaned_data[['release_day','release_month','release_year']].astype('str')

In [ ]:
cleaned_data.dtypes

### Now convert the release day, month, and year to day-month-year

In [ ]:
cleaned_data['date'] = pd.to_datetime(cleaned_data['release_year'] + '-' +  cleaned_data['release_month'] + '-'+ cleaned_data['release_day'])
cleaned_data['date']

In [ ]:
cleaned_data.head(4)

### convert the release_month to the month name

In [ ]:
cleaned_data['Release_Month_name'] = cleaned_data.date.dt.month_name()
cleaned_data.head(4)

### Getting the release_week_day

In [ ]:

cleaned_data['Release_week_day'] = cleaned_data.date.dt.day_name()

In [ ]:
## viewing the dataframe
cleaned_data.head(4)

In [ ]:
## column names
cleaned_data.columns

### Droping the release day, month since we created a new column of day-month-year and movie_rank because its just a count

In [ ]:
cleaned_reformed = cleaned_data.drop(columns = ['release_day'])
cleaned_reformed = cleaned_reformed.drop(columns=['movie_rank'])
cleaned_reformed = cleaned_reformed.drop(columns=['release_month'])
cleaned_reformed.head(6)

In [ ]:
## renaming the description column to Synopsis
cleaned_reformed.rename(columns={'description':'Synopsis'},inplace=True)
cleaned_reformed.head(4)


## **Question 2 Part One**

In [ ]:
## Most common genres
genre_data = cleaned_reformed.groupby(['genres'],as_index=None,observed=True)['genres'].agg({'count'})
genre_data

In [ ]:
## Extracting  most released genres
# The most released genres are below
genre_data.loc[genre_data['count'] >= 7][['genres','count']]

#### Hence the top released genres are `Drama`, `crime,Drama`, `Biography, Drama, History`,`Animation, Adventure, Comedy` and `Action, Adventure, Sci-Fi`

In [ ]:
## A countplot showing the most release'd genres
plt.figure(figsize=(30,10))
sns.countplot(data=cleaned_reformed,x='genres')
plt.xticks(rotation = 90)
plt.title('a countplot showing the most released genres'.title())

In [ ]:
# Split and count genre occurrences
all_genres = cleaned_reformed['genres'].dropna().str.split(', ').sum()
genre_counts = Counter(all_genres)

# Identify the most frequent genre
most_frequent_genre = genre_counts.most_common(1)[0]
most_frequent_genre


In [ ]:
# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='viridis')
wordcloud.generate_from_frequencies(genre_counts)

# Plot word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('IMDb Top Movie Genres')
plt.show()

## **Question 2 Part 2**

In [ ]:
average_rating = cleaned_reformed.agg({'rating':'mean'})
average_rating

## **Question 2 Part 3**

In [ ]:
## converting the release_year to integer
cleaned_reformed['release_year'] = cleaned_reformed['release_year'].astype(int)

In [ ]:

# Set visualization style
sns.set_style("whitegrid")

# Create subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# 1. Genre Distribution
sns.barplot(x=list(genre_counts.keys()), y=list(genre_counts.values()), ax=axes[0], palette="viridis")
axes[0].set_title("Genre Distribution in IMDb Top Movies")
axes[0].set_ylabel("Number of Movies")
axes[0].tick_params(axis='x', rotation=45)

# 2. IMDb Rating Distribution
sns.histplot(cleaned_reformed["rating"], bins=10, kde=True, ax=axes[1], color="blue")
axes[1].set_title("IMDb Rating Distribution")
axes[1].set_xlabel("IMDb Rating")
axes[1].set_ylabel("Count")

# 3. Movie Release Decades
cleaned_reformed["release_decade"] = (cleaned_reformed["release_year"] // 10) * 10  # Group years into decades
decade_counts = cleaned_reformed["release_decade"].value_counts().sort_index()
sns.barplot(x=decade_counts.index.astype(str), y=decade_counts.values, ax=axes[2], palette="coolwarm")
axes[2].set_title("Number of Top Movies by Release Decade")
axes[2].set_xlabel("Decade")
axes[2].set_ylabel("Number of Movies")



# Adjust layout and display
plt.tight_layout()
plt.show()


### **Key Insights from IMDb Top Movies Analysis**  

 **1. Genre Distribution**  
   - **Drama** is the most dominant genre, appearing in most top-rated movies.  
   - Other frequently occurring genres include **Crime, Action, and Biography**, showing a preference for intense storytelling and real-life inspirations.  

 **2. IMDb Rating Distribution**  
   - The **average rating is 8.31**, indicating that IMDb's Top Movies are highly rated.  
   - Most movies have ratings between **8.0 and 9.0**, with very few falling below 8.0.  


 **3. Movie Release Decades**  
   - The **1990s and 2000s** produced the highest number of top-rated movies.  
   - Classic films from earlier decades (e.g., **1970s and 1980s**) still hold strong rankings.  
   - Recent decades (**2010s and 2020s**) have fewer movies in the Top List, possibly due to evolving audience preferences or ranking criteria.  



In [ ]:
## sorting the data by release_year in descending order
Rate_sorted_data = cleaned_reformed.sort_values(by='rating',ascending=False)
Rate_sorted_data.head(6)

## **WEEK DAYS AND MONTHS IN WHICH MOVIES MOST RELEASED**

- Graphically

In [ ]:
all_week_days = cleaned_reformed['Release_week_day'].dropna().str.split(', ').sum()
day_counts = Counter(all_week_days)

# Identify the most frequent genre
most_all_week_days = genre_counts.most_common(1)[0]
day_counts

In [ ]:
## Days in which most movies are released
wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='viridis')
wordcloud.generate_from_frequencies(day_counts)

# Plot word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Week Days In Which Movies Are Mostly Released')
plt.show()


##### We see that most movies are released on Friday,Wednesday and Thurseday

In [ ]:
cleaned_reformed.columns

- Statistics

In [ ]:
cleaned_reformed.groupby(['Release_week_day'],as_index=False)['Release_week_day'].agg({'count'})


### We see that most movies are released on Friday,Wednesday and Thurseday

## **The months**

In [ ]:
cleaned_reformed.columns

In [ ]:
all_months = cleaned_reformed['Release_Month_name'].dropna().str.split(', ').sum()
month_counts = Counter(all_months)

# Identify the most frequent day_counts
most_all_months = month_counts.most_common(1)[0]
month_counts

In [ ]:
## Months in which movies are mostly released
wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='viridis')
wordcloud.generate_from_frequencies(month_counts)

# Plot word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Months in Which Movies Are Mostly Released')
plt.show()


### **Most movies are released in December, November and june**

In [ ]:
cleaned_reformed.groupby(['Release_Month_name'],as_index=False)['Release_Month_name'].agg({'count'})


#### **Most movies are released in December, November and june**

# **Using statistical analysis and visualization techniques**

In [ ]:
## checking for numerical columns
numerical_cols = cleaned_reformed.select_dtypes(include=['int64', 'float64'])
numerical_cols.columns

In [ ]:

# Function to convert time format to minutes
def convert_to_minutes(length):
    if isinstance(length, str):  # Ensure it's a string before processing
        length = length.strip()  # Remove extra spaces
        if ":" in length:  # Handle HH:MM:SS or MM:SS formats
            parts = list(map(int, length.split(":")))  # Split into components
            if len(parts) == 3:  # If format is HH:MM:SS
                return parts[0] * 60 + parts[1]  # Convert hours to minutes
            elif len(parts) == 2:  # If format is MM:SS
                return parts[0]  # Keep minutes only
        elif "min" in length:  # Handle '123min' format
            return int(length.replace("min", "").strip())
    return None  # Return None for invalid values

# Apply the function to the column
cleaned_reformed["movie_length"] = cleaned_reformed["movie_length"].apply(convert_to_minutes)

# Check if conversion is successful
print(cleaned_reformed["movie_length"].head())


In [ ]:
## checking for numerical columns
numerical_cols = cleaned_reformed.select_dtypes(include=['int64', 'float64'])
numerical_cols.columns

In [ ]:
## using visualization techniques



# Compute correlation matrix
correlation_matrix = numerical_cols.corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Movie Parameters")
plt.show()

# Identify highest correlations with IMDb Rating
correlation_with_rating = correlation_matrix["rating"].sort_values(ascending=False)
print("Top Correlated Features with IMDb Rating:\n", correlation_with_rating)

# Scatter plots for key correlated features
key_features = ["movie_length", "vote_count", "release_year"]  
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, feature in enumerate(key_features):
    sns.scatterplot(x=numerical_cols[feature], y=numerical_cols["rating"], ax=axes[i], alpha=0.6)
    axes[i].set_title(f"IMDb Rating vs {feature.capitalize()}")
    axes[i].set_xlabel(feature.capitalize())
    axes[i].set_ylabel("IMDb Rating")

plt.tight_layout()
plt.show()


In [ ]:
## pairplot
sns.pairplot(cleaned_reformed)


In [ ]:
cleaned_reformed.describe()


## **Satistical analysis**

In [ ]:
## convert the Box Office Gross into integer
cleaned_reformed["Box Office Gross"] = cleaned_reformed["Box Office Gross"].apply(lambda x: int(x.replace("$", "").replace(",", "").strip()) if isinstance(x, str) and x.replace("$", "").replace(",", "").strip().isdigit() else None)


In [ ]:
cleaned_reformed.head(5)

In [ ]:
# reformed columns
cleaned_reformed.columns

In [ ]:
# basic statistics
boxoffice_stats = cleaned_reformed["Box Office Gross"].describe()
print(boxoffice_stats)


### Box Plot to Identify Outliers

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(x=cleaned_reformed["Box Office Gross"], color="blue")
plt.xlabel("Box Office Gross ($)")
plt.title("Box Office Gross Boxplot showing the movies that earned way more  than others")
plt.show()


###  Outliers are showing the movies that earned way more  than others.

## Top 10 Highest-Grossing Movies

In [ ]:
# Display top 10 highest-grossing movies
top_10_movies = cleaned_reformed.sort_values(by="Box Office Gross", ascending=False)[["title", "Box Office Gross"]].head(10)
print(top_10_movies)


In [ ]:
cleaned_reformed.columns

In [ ]:
# # Display top 10 highest-grossing movies
# print("Top 10 highest-grossing movies:")

# for index, row in top_10_movies.iterrows():
#     print(f"{index+1}. {row['title']} - ${row['Box Office Gross']:,}")
#     print()
#     # Print release year and genre
#     print(f"Release Year: {row['release_year']}")
#     print(f"Genres: {row['genres']}")
#     print()  # Empty line for better readability
#     # Print a line separator
#     print("-" * 50)
#     print()
#     # Print a line separator
#     print()
#     # Calculate and print average box office gross per genre
#     genre_avg_gross = cleaned_reformed.groupby("genres")["Box Office Gross"].mean().sort_values(ascending=False)
#     print("Average Box Office Gross per Genre:")
#     for genre, avg_gross in genre_avg_gross.items():
#         print(f"{genre}: ${avg_gross:,.2f}")
#         print()  # Empty line for better readability
#         # Print a line separator
#         print()
        
                                                                       
        


In [ ]:

# Create a dictionary: {title: Box Office Gross}
title_gross_dict = dict(zip(cleaned_reformed["title"], cleaned_reformed["Box Office Gross"]))
# Normalize values for better visualization
max_gross = max(title_gross_dict.values())
normalized_dict = {title: (gross / max_gross) * 100 for title, gross in title_gross_dict.items()}

# Generate Word Cloud
wordcloud = WordCloud(width=1000, height=600, background_color="black", colormap="coolwarm",
                      max_words=100, relative_scaling=0.5).generate_from_frequencies(normalized_dict)

# Display Word Cloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")  # Hide axes
plt.title("Word Cloud of Movie Titles Based on Box Office Gross", fontsize=14)
plt.show()


In [ ]:
# Display top 10 highest-grossing movies
top_10_movies = cleaned_reformed.sort_values(by="Box Office Gross", ascending=False)[["title", "Box Office Gross"]].head(10)
print(top_10_movies)